In [1]:
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset

2022-08-09 18:42:55.010965: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
strongbuys = list()
holddowns = list()

for idx in range(332, 415):
    try:
        file = open(f"./data/strongbuy/strongbuy{idx}.txt", "r")
        strings = file.readlines()
        strings = "".join(strings)
        string_list = strings.split("!@#$")
        for string in string_list:
            strongbuys.append(string)
        file.close()
    except:
        pass

for idx in range(212, 265):
    try:
        file = open(f"./data/holddown/holddown{idx}.txt", "r")
        strings = file.readlines()
        strings = "".join(strings)
        string_list = strings.split("!@#$")
        for string in string_list:
            holddowns.append(string)
        file.close()
    except:
        pass

print(len(strongbuys))
print(len(holddowns))

189
90


In [3]:
import pandas as pd

df = pd.DataFrame(columns=("document", "labels"))

for doc in strongbuys: df = df.append({"document":doc, "labels":1}, ignore_index=True)
for doc in holddowns:  df = df.append({"document":doc, "labels":0}, ignore_index=True)

In [4]:
import re

nonpreprocesses = list(df["document"])
docs = list(df["document"])

for idx, doc in enumerate(docs):
    doc = re.sub(r"\s+", " ", doc)    
    docs[idx] = doc

for idx, (nonpreprocess, doc) in enumerate(zip(nonpreprocesses, docs)):
    if idx == 3:
        break

    print(nonpreprocess)
    print("-" * 20)
    print(doc)
    print("=" * 40)

df["document"] = docs
df

투자의견 Strong Buy, 목표주가 50,000원 유지
1Q16 실적 기대치 상회, 고객사 내 점유율 확대 효과
2Q16 영업이익 146억원으로 성장세 지속
LG디스플레이 내 TV패널 점유율 확대, OLED패널 독점 공급에 따른 성장세 지속 전망
1Q16 매출액 1,590억원(+83% YoY), 영업이익 149억원(+52% YoY) , 기대치 상회
당사 예상과 같이 고객사 내 점유율 상승과 이에 따른 수익성 개선 효과가 나타났음
스마트폰과 테블릿PC 내 점유율 확대로 COG 부문의 실적 호조
TV패널 내 점유율 확대로 COF, TCON, PMIC의 실적 호조
판관비 내 일회성 비용(제품손실보증비 15억원) 제외하면 실질 영업이익은 165억원 수준
2Q16 매출액 1,548억원(+24% YoY), 영업이익 146억원(+17% YoY) , 성장 지속 전망
스마트폰과 테블릿PC 수요 감소로 COG부문의 실적 감소 예상되나,
TV패널 내 점유율 확대가 이를 상쇄하며 실적 성장세 지속시킬 전망
--------------------
투자의견 Strong Buy, 목표주가 50,000원 유지 1Q16 실적 기대치 상회, 고객사 내 점유율 확대 효과 2Q16 영업이익 146억원으로 성장세 지속 LG디스플레이 내 TV패널 점유율 확대, OLED패널 독점 공급에 따른 성장세 지속 전망 1Q16 매출액 1,590억원(+83% YoY), 영업이익 149억원(+52% YoY) , 기대치 상회 당사 예상과 같이 고객사 내 점유율 상승과 이에 따른 수익성 개선 효과가 나타났음 스마트폰과 테블릿PC 내 점유율 확대로 COG 부문의 실적 호조 TV패널 내 점유율 확대로 COF, TCON, PMIC의 실적 호조 판관비 내 일회성 비용(제품손실보증비 15억원) 제외하면 실질 영업이익은 165억원 수준 2Q16 매출액 1,548억원(+24% YoY), 영업이익 146억원(+17% YoY) , 성장 지속 전망 스마트폰과 테블릿PC 수요 감소로 COG부문의 실적 감소 예상되나, TV패널 내 점유

,document,labels
0,"투자의견 Strong Buy, 목표주가 50,000원 유지 1Q16 실적 기대치 상...",1
1,▶ Highlight 2Q16 어닝쇼크 때와는 정반대의 어닝서프라이즈. 진에어 홀로...,1
2,"▶ In detail - 진에어: 매출액 2193억원(YoY +48%, QoQ +...",1
3,"숨겨진진주 3Q14 Review : 매출 3469 억원 , 영업이익 428억원 (O...",1
4," 4Q14 Preview: 매출 3,554 억원 영업이익 438억원 (OPM 1...",1
...,...,...
274,실적 해석 및 시사점 견조한 실적 개선세 지속: 1Q 양호한 실적은 1) RC값 ...,0
275,한국항공우주 (047810) 코로나19 여파로 상반기는 저공비행 투자의견 시장수익률...,0
276,"1Q21 Review 코로나19 여파로 기체부품, 완제기 수출의 매출 감소 - 1...",0
277,"롯데케미칼 011170 회복을 논하기에는 이른 시점  4Q19 영업이익 1,426...",0


In [5]:
from sklearn.model_selection import train_test_split

train_data, valid_data = train_test_split(df, test_size=0.2, random_state=42, stratify=df[["labels"]])
print(train_data[train_data["labels"]==1].count()["labels"])
print(train_data[train_data["labels"]==0].count()["labels"])
print(f"1 : {train_data[train_data['labels']==0].count()['labels'] / train_data[train_data['labels']==1].count()['labels']:.3f}")
print("=" * 30)
print(valid_data[valid_data["labels"]==1].count()["labels"])
print(valid_data[valid_data["labels"]==0].count()["labels"])
print(f"1 : {valid_data[valid_data['labels']==0].count()['labels'] / valid_data[valid_data['labels']==1].count()['labels']:.3f}")

151
72
1 : 0.477
38
18
1 : 0.474


In [6]:
train_data.to_csv("./data/train_data.csv", index=False)
valid_data.to_csv("./data/valid_data.csv", index=False)

## 분기점

In [7]:
kb_albert_model_path = "./model/kb-albert-char-base-v2"
albert = AutoModel.from_pretrained(kb_albert_model_path)
tokenizer = AutoTokenizer.from_pretrained(kb_albert_model_path)

tokenizer.truncation_side = "left"

Some weights of the model checkpoint at ./model/kb-albert-char-base-v2 were not used when initializing AlbertModel: ['predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.bias', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
from datasets import load_dataset

MAX_LEN = 512

def tokenized_fn(data):
    outputs = tokenizer(data["document"], padding="max_length", max_length=MAX_LEN, truncation=True)
    outputs["labels"] = data["labels"]
    return outputs

train_dataset = load_dataset("csv", data_files="./data/train_data.csv")["train"]
valid_dataset = load_dataset("csv", data_files="./data/valid_data.csv")["train"]

train_dataset = train_dataset.map(tokenized_fn, remove_columns=["document"])
valid_dataset = valid_dataset.map(tokenized_fn, remove_columns=["document"])

Using custom data configuration default-5219091d43a73f9c


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/piai/.cache/huggingface/datasets/csv/default-5219091d43a73f9c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-86442a4fdb4058a0


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/piai/.cache/huggingface/datasets/csv/default-86442a4fdb4058a0/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?ex/s]

  0%|          | 0/56 [00:00<?, ?ex/s]

In [9]:
# list_input_ids = list()
# list_token_type_ids = list()
# list_attention_mask = list()
# for idx, doc in enumerate(df["document"]):
#     tokenized_doc = tokenizer(doc, padding="max_length", max_length=MAX_LEN, truncation=True)
#     list_input_ids.append(tokenized_doc["input_ids"])
#     list_token_type_ids.append(tokenized_doc["token_type_ids"])
#     list_attention_mask.append(tokenized_doc["attention_mask"])

# df["input_ids"] = list_input_ids
# df["token_type_ids"] = list_token_type_ids
# df["attention_mask"] = list_attention_mask
# df = df[["document", "input_ids", "token_type_ids", "attention_mask", "labels"]]
# df.head()

In [10]:
class ClassificationHead(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # self.dense1 = torch.nn.Linear(768, 3072)
        self.dropout = torch.nn.Dropout(0.25)
        # self.dense2 = torch.nn.Linear(3072, 768)
        self.out_proj = torch.nn.Linear(768, 2)
    
    def forward(self, features):
        # 보통 분류기에선 start 토큰에 분류 결과를 담음
        x = features[:, 0, :]    # take <s> token (equiv. to [CLS])
        x = x.reshape(-1, x.size(-1))
        x = self.dropout(x)
        
#         x = self.dense1(x)
#         x = torch.relu(x)
#         x = self.dropout(x)
        
#         x = self.dense2(x)
#         x = torch.tanh(x)
#         x = self.dropout(x)
        x = self.out_proj(x)
        return x

class AInalyst(torch.nn.Module):
    def __init__(self, pretrained_model):
        super(AInalyst, self).__init__()
        self.pretrained = pretrained_model
        self.classifier = ClassificationHead()
    
    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.pretrained(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # labels=labels
        )
        self.labels = labels
        logits = self.classifier(outputs["last_hidden_state"])
        # prob = torch.nn.functional.softmax(logits, dim=-1)
        
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            return logits, loss
        else:
            return logits

In [11]:
# def crossvalid(model=None, criterion=None, optimizer=None, dataset=None, k_fold=5):
#     total_size = len(dataset)
#     fraction = 1/k_fold
#     seg = int(total_size * fraction)
    
#     for idx in range(k_fold):
#         trill = 0
#         trilr = idx * seg
#         vall = trill
#         valr = i * seg + seg
#         trirl = valr
#         trirr = total_size
        
        
        

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weight = torch.tensor([0.5, 0.5]).to(device)
loss_fct = torch.nn.CrossEntropyLoss(weight=weight)

In [13]:
model = AInalyst(pretrained_model=albert)
model.to(device)
model = torch.nn.DataParallel(model)
isParallel = True

In [14]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [15]:
from transformers import DataCollatorWithPadding

batch_size = 2

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    sampler = torch.utils.data.RandomSampler(train_dataset),
    batch_size = batch_size,
    collate_fn = data_collator,
)

validation_dataloader = torch.utils.data.DataLoader(
    valid_dataset,
    sampler = torch.utils.data.SequentialSampler(valid_dataset),
    batch_size = batch_size,
    collate_fn = data_collator,
)

In [16]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [17]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [18]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

epochs = 5
for epoch in range(epochs):
    print(f"====== Epoch {epoch+1}/{epochs} ======")
    print("Training...")
    
    total_train_loss = 0
    model.train()
    
    for step, batch in enumerate(train_dataloader):
        batch_input_ids = batch["input_ids"].to(device)
        batch_attention_mask = batch["attention_mask"].to(device)
        batch_labels = batch["labels"].to(device)
        
        model.zero_grad()
        
        logits, loss = model(
            input_ids = batch_input_ids,
            attention_mask = batch_attention_mask,
            labels = batch_labels,
        )
        
        # # output 모양 출력해보기
        # outputs = model(
        #     input_ids = batch_input_ids,
        #     attention_mask = batch_attention_mask,
        #     labels = batch_labels,
        # )
        # print("print outputs : ")
        # print(outputs)
        # print(outputs.last_hidden_state.shape)
        # break
    
        if isParallel:
            loss = loss.mean()

        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
    
        if step % 1000 == 0 and not step == 0:
            print("step : {:>5,} of {:>5,} loss: {:.5f}".format(step, len(train_dataloader), loss.item()))
    
    avg_train_loss = total_train_loss / len(train_dataloader)
    print()
    print(" Average training loss: {0:.5f}".format(avg_train_loss))
    
    # Validation
    print()
    print("Running Validation...")
    
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    
    for step, batch in enumerate(validation_dataloader):
        batch_input_ids = batch["input_ids"].to(device)
        batch_attention_mask = batch["attention_mask"].to(device)
        batch_labels = batch["labels"].to(device)
    
        with torch.no_grad():
            logits, loss = model(
                input_ids = batch_input_ids,
                attention_mask = batch_attention_mask,
                labels = batch_labels,
            )
            
            if isParallel:
                loss = loss.mean()
            
            total_eval_loss += loss.item()
            logits = logits.detach().cpu().numpy()
            label_ids = batch_labels.to("cpu").numpy()
            total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("Accuracy: {0:.5f}".format(avg_val_accuracy))

====== Epoch 1/5 ======
Training...


/home/piai/anaconda3/envs/iml/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '



 Average training loss: 0.62136

Running Validation...
Accuracy: 0.78571
====== Epoch 2/5 ======
Training...

 Average training loss: 0.39893

Running Validation...
Accuracy: 0.75000
====== Epoch 3/5 ======
Training...

 Average training loss: 0.18474

Running Validation...
Accuracy: 0.83929
====== Epoch 4/5 ======
Training...

 Average training loss: 0.05745

Running Validation...
Accuracy: 0.87500
====== Epoch 5/5 ======
Training...

 Average training loss: 0.02522

Running Validation...
Accuracy: 0.89286
